<a href="https://colab.research.google.com/github/albertiluis/albertiluis/blob/main/ajustePTHP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from tkinter import W
from eppy.results import readhtml
from eppy import modeleditor
from eppy.modeleditor import IDF
from eppy.results import fasthtml

import pprint

iddfile = "D:\ProgramFiles\EnergyPlusV8-7-0\Energy+.idd"
idf = "D:\DADOS\Desktop\Python\idf\conteiner\conteiner.idf"
table = "D:\DADOS\Desktop\Python\idf\conteiner\conteinerTable.html"

IDF.setiddname(iddfile)
idf1 = IDF(idf)
pp = pprint.PrettyPrinter()

pthp = idf1.idfobjects['HVACTEMPLATE:ZONE:PTHP']
qtdePTHP = len(pthp)

# Função para ajuste do COP de Heating
def ajusteHeatingCOP():
    filehandle = open(table, 'r')
    heatingOutput = fasthtml.tablebyname(filehandle, "Heating Coils")
    heatingOutputTable = heatingOutput[1]

    for n in range(len(heatingOutputTable)+1):
        if n % 2 != 0:
            heatingBtu = round(heatingOutputTable[n][3] * 3.4121, 2)
            print("Total Heating Capacity [W]: " + str(heatingOutputTable[n][3]))
            print("Total Heating Capacity [Btu/h]: " + str(heatingBtu))

            if heatingBtu <= 7000:
                newHeatingCOP = 3.33
                pthp[n].Heat_Pump_Heating_Coil_Gross_Rated_COP = newHeatingCOP
                print("New Heating COP 7000: " + str(newHeatingCOP))

            if heatingBtu >= 15000:
                newHeatingCOP = 2.92
                pthp[n].Heat_Pump_Heating_Coil_Gross_Rated_COP = newHeatingCOP
                print("New Heating COP 15000: " + str(newHeatingCOP))

            if 7000 < heatingBtu < 15000:
                newHeatingCOP = round((3.7 - (0.052 * (heatingBtu/1000))), 1)
                pthp[n].Heat_Pump_Heating_Coil_Gross_Rated_COP = newHeatingCOP
                print("New Heating COP btu: " + str(newHeatingCOP))


# Função para ajuste do COP de Cooling
def ajusteCoolingCOP():
    filehandle = open(table, 'r')
    coolingOutput = fasthtml.tablebyname(filehandle, "Cooling Coils")
    coolingOutputTable = coolingOutput[1]

    for n in range(qtdePTHP):
        coolingBtu = round(coolingOutputTable[n+1][3] * 3.4121, 2)
        print("Total Cooling Capacity [W]: " + str(coolingOutputTable[n+1][3]))
        print("Total Cooling Capacity [Btu/h]: " + str(coolingBtu))

        if coolingBtu <= 7000:
            newCoolingCOP = 3.39
            pthp[n].Cooling_Coil_Gross_Rated_COP = newCoolingCOP
            print("New Cooling COP 7000: " + str(newCoolingCOP))

        if coolingBtu >= 15000:
            newCoolingCOP = 2.70
            pthp[n].Cooling_Coil_Gross_Rated_COP = newCoolingCOP
            print("New Cooling COP 15000: " + str(newCoolingCOP))

        if 7000 < coolingBtu < 15000:
            newCoolingCOP = round((14 - (0.300 * (coolingBtu/1000)))/3.51, 1)
            pthp[n].Cooling_Coil_Gross_Rated_COP = newCoolingCOP
            print("New Cooling COP btu: " + str(newCoolingCOP))
            

# Função para ajustar pressão conformo a nova FanPower definida
def ajustePRESSAO():
    filehandle = open(table, 'r')
    fansOutput = fasthtml.tablebyname(filehandle, "Fans")
    fansOutputTable = fansOutput[1]

    for n in range(qtdePTHP):
        maxAirFlow = fansOutputTable[n+1][4]
        fanPressure = fansOutputTable[n+1][3]
        fanPower = fansOutputTable[n+1][5]

        maxAirFlowCFM = round(maxAirFlow * 2118.88, 2)        
        wcfm = round(fanPower / maxAirFlowCFM, 2)
        dif = round(0.3 / wcfm, 2)
        newPressure = round(dif * fanPressure, 2)

        pthp[n].Supply_Fan_Delta_Pressure = newPressure

        print("New Pressure: " + str(newPressure))


# ajusteCoolingCOP()
ajusteHeatingCOP()
# ajustePRESSAO()

# pp.pprint(pthp)

# idf1.saveas('HVACajusteMOOV.idf')
# # print("SAVE")